In [1]:
from langchain.memory.chat_message_histories import ZepChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.schema import HumanMessage, AIMessage
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool
from uuid import uuid4
import pprint

In [2]:
# Set this to your Zep server URL
ZEP_API_URL = "http://localhost:8000"

session_id = str(uuid4())  # This is a unique identifier for the user

In [3]:
# Load your OpenAI key from a .env file
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
search = WikipediaAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to search online for answers. You should ask targeted questions"
    ),
]

# Set up Zep Chat History
zep_chat_history = ZepChatMessageHistory(
    session_id=session_id,
    url=ZEP_API_URL,
)

# Use a standard ConversationBufferMemory to encapsulate the Zep chat history
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=zep_chat_history
)

# Initialize the agent
llm = OpenAI(temperature=0)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

In [5]:
# Preload some messages into the memory. The default message window is 12 messages. We want to push beyond this to demonstrate auto-summarization.
test_history = [
    {"role": "human", "content": "Who was Octavia Butler?"},
    {
        "role": "ai",
        "content": (
            "Octavia Estelle Butler (June 22, 1947 – February 24, 2006) was an American"
            " science fiction author."
        ),
    },
    {"role": "human", "content": "Which books of hers were made into movies?"},
    {
        "role": "ai",
        "content": (
            "The most well-known adaptation of Octavia Butler's work is the FX series"
            " Kindred, based on her novel of the same name."
        ),
    },
    {"role": "human", "content": "Who were her contemporaries?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler's contemporaries included Ursula K. Le Guin, Samuel R."
            " Delany, and Joanna Russ."
        ),
    },
    {"role": "human", "content": "What awards did she win?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur"
            " Fellowship."
        ),
    },
    {
        "role": "human",
        "content": "Which other women sci-fi writers might I want to read?",
    },
    {
        "role": "ai",
        "content": "You might want to read Ursula K. Le Guin or Joanna Russ.",
    },
    {
        "role": "human",
        "content": (
            "Write a short synopsis of Butler's book, Parable of the Sower. What is it"
            " about?"
        ),
    },
    {
        "role": "ai",
        "content": (
            "Parable of the Sower is a science fiction novel by Octavia Butler,"
            " published in 1993. It follows the story of Lauren Olamina, a young woman"
            " living in a dystopian future where society has collapsed due to"
            " environmental disasters, poverty, and violence."
        ),
    },
]

for msg in test_history:
    zep_chat_history.append(
        HumanMessage(content=msg["content"])
        if msg["role"] == "human"
        else AIMessage(content=msg["content"])
    )

# Inspect the Zep memory
**Note the summary, and that the history has been enriched with entities, token counts, UUIDs, and timestamps.**

Summaries are biased towards the most recent messages.

In [6]:
def print_messages(messages):
    for m in messages:
        print(m.dict())


print(zep_chat_history.zep_summary)
print("\n")
print_messages(zep_chat_history.zep_messages)

The human asks about Octavia Butler and the AI explains that she was an American science fiction author. The human then inquires about which movies were adapted from her books and the AI mentions the FX series Kindred. The human also asks about Butler's contemporaries and the AI provides names such as Ursula K. Le Guin, Samuel R. Delany, and Joanna Russ.


{'role': 'human', 'content': 'What awards did she win?', 'uuid': '6666a9d0-c24c-4784-b810-51a5bd90c44b', 'created_at': '2023-05-26T17:28:26.31749Z', 'token_count': 8, 'metadata': {'system': {'entities': []}}}
{'role': 'ai', 'content': 'Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur Fellowship.', 'uuid': '928e4307-3f2d-425e-9ce2-2eece865cf99', 'created_at': '2023-05-26T17:28:26.324468Z', 'token_count': 21, 'metadata': {'system': {'entities': [{'Label': 'PERSON', 'Matches': [{'End': 14, 'Start': 0, 'Text': 'Octavia Butler'}], 'Name': 'Octavia Butler'}, {'Label': 'WORK_OF_ART', 'Matches': [{'End': 33, 'Start': 19

In [7]:
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(zep_chat_history.zep_messages[1].dict())

{   'content': 'Octavia Butler won the Hugo Award, the Nebula Award, and the '
               'MacArthur Fellowship.',
    'created_at': '2023-05-26T17:28:26.324468Z',
    'metadata': {   'system': {   'entities': [   {   'Label': 'PERSON',
                                                      'Matches': [   {   'End': 14,
                                                                         'Start': 0,
                                                                         'Text': 'Octavia '
                                                                                 'Butler'}],
                                                      'Name': 'Octavia Butler'},
                                                  {   'Label': 'WORK_OF_ART',
                                                      'Matches': [   {   'End': 33,
                                                                         'Start': 19,
                                                                         'Te

In [9]:
agent_chain.run(
    input="Recommend science fiction books I should read that are similar to Octavia Butler's Parable of the Sower. Research online."
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: Recommend science fiction books similar to Octavia Butler's Parable of the Sower
Observation: Page: Dune (novel)
Summary: Dune is a 1965 epic science fiction novel by American author Frank Herbert, originally published as two separate serials in Analog magazine. It tied with Roger Zelazny's This Immortal for the Hugo Award in 1966 and it won the inaugural Nebula Award for Best Novel. It is the first installment of the Dune saga. In 2003, it was described as the world's best-selling science fiction novel.Dune is set in the distant future amidst a feudal interstellar society in which various noble houses control planetary fiefs.  It tells the story of young Paul Atreides, whose family accepts the stewardship of the planet Arrakis. While the planet is an inhospitable and sparsely populated desert wasteland, it is the only source of melange, or "spice", a drug that extends life and en

"Based on your request, some recommended science fiction books similar to Octavia Butler's Parable of the Sower include The Left Hand of Darkness by Ursula K. Le Guin, The Dispossessed by Ursula K. Le Guin, The Female Man by Joanna Russ, Dune by Frank Herbert, and The Forever War by Joe Haldeman."

In [10]:
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(zep_chat_history.zep_messages[-1].dict())

{   'content': 'Based on your request, some recommended science fiction books '
               "similar to Octavia Butler's Parable of the Sower include The "
               'Left Hand of Darkness by Ursula K. Le Guin, The Dispossessed '
               'by Ursula K. Le Guin, The Female Man by Joanna Russ, Dune by '
               'Frank Herbert, and The Forever War by Joe Haldeman.',
    'created_at': '2023-05-26T17:31:36.433237Z',
    'metadata': {   'system': {   'entities': [   {   'Label': 'ORG',
                                                      'Matches': [   {   'End': 89,
                                                                         'Start': 73,
                                                                         'Text': 'Octavia '
                                                                                 "Butler's"}],
                                                      'Name': 'Octavia '
                                                              "B